# Задание 2. Распознавание именованных сущностей

Во втором практическом задании мы будем решать задачу распознавания именованных сущностей. Будем различать три вида сущностей:
- Фамилия, имя и отчство,
- Названия городов,
- Наименование контрагента

![](img/2_1.png)

Загрузим данные. Данные представляют собой те же обращения пользователей, которые мы использовали в первом практическом задании, только теперь мы будем анализировать каждое слово обращения, а не все обращение целиком:

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from pandas import Series, DataFrame


df = pd.read_excel("ner_data_with_flowers.xlsx")
df["words"] = df["words"].astype(str)
df.head()

Поля таблицы имеют три возможных значения:
- $O$ - слово не является именованной сущностью
- $н$ - начало именованной сущности
- $п$ - продолжение именованной сущности

Рассмотрим примеры каждой именованной сущности:

- Контрагент:

In [ ]:
df[df["Контрагент"] != "О"][:10]

- Фамилия, имя и отчество:

In [ ]:
df[df["ФИО"] != "О"][:10]

- Город:

In [ ]:
df[df["Город"] != "О"][:9]

Обращения разделеены вспомогательным символом "----"

In [ ]:
df[8:13]

Это позволяет нам добавить информацию о номере обращения, слово из которого рассматривается:

In [ ]:
df["sent_number"] = (df["words"] == "---").astype(int).cumsum()
df[8:13]

Первым делом нам необходимо преобразовать данные к виду, в котором они будут использоваться моделями (как и в первом практическом задании). Как видно, данные уже приведены к нижнему регистру и из них удалены знаки препинания. Так что в предобрабока будет включать в себя лишь приведение слов к нормальной форме путем лемматизации. Как и раньше, предпочтителен подход, при котором мы сперва лемматизируем все уникальные слова и только потом производим замены слов на их лемматизированные аналоги:

In [ ]:
from pymorphy2 import MorphAnalyzer

lemmatizer = MorphAnalyzer()
to_normal_form = { word : lemmatizer.normal_forms(word) for word in df["words"].unique() }
df["words"] = df["words"].apply(lambda x : to_normal_form[x][0])

In [ ]:
df.head()

Мы будем использовать следующие методы распознавания именованных сущностей:
- Неструктурированные методы
    - Базовый подход, который будет заключаться в запоминании меток всех слов, которые встрчались в обучающем множестве
    - Стадартандартные методы машинного обучения ($RandomForest$ и $LogisticRegression$)
- Структурированные методы  
    - Рекуррентные нейронные сети

В качестве точности уже неинформативно использовать метрику $accuarcy$, поскольку выборка несбалансирована (мы можем получать высокую точность, предсказывая константную метку $O$, но это плохое предсказание). Поэтому в качестве метрики мы будем использовать три другие величины.

Пусть мы решаем бинарную задачу с метками $0$ и $1$. Рассмотрим следующие метрики:
- **Precision (точность)** - это часть правильно размеченных объектов среди объектов с истиной меткой $1$
- **Recall (полноста)** - это часть правильно размеченных объектов среди объектов с предсказанной меткой $1$
- **F1 мера** - метрика, объединяющая **Precision** и **Recall** по следующему правилу:
$$
F_1 = \frac{2 \cdot Precision \cdot Recall}{Precision + Recall}
$$

Задача моделей: получить высокую точность по каждой метрике для каждой из меток: $н$ и $п$. 

# Неструктурированные методы

Начнем с неструктурированных методов. Первоначально необходимо разбить слова всех обращений на обучающую и отложенную выборки:

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_dev = train_test_split(df, test_size=0.2)
print("===== полученные размерности =====")
print("df_train.shape:", df_train.shape)
print("df_dev.shape:", df_dev.shape)

Теперь мы можем приступить к реализации методов.

## Базовый подход

Базовый подход заключается просто в запоминании тегов для все слов из тренировочного множества. Более подробно:
- Для каждого уникального слова храним словарь, в котором записываем, сколько раз в тренировочном множестве встречались метки $О$, $н$ и $п$
- Запоминаем самую частотную метку для каждого слова
- Во время тестирования для каждогослова выдаем самую частотную метку

Реализуем класс, который будет обучаться и производить предсказания описанным выше способом:

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from collections import defaultdict

class BasicModel(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y):
        words = Series(X).unique().tolist()
        self.vocabulary = {word : {"О" : 0, "н" : 0, "п" : 0} for word in words}
        for word, entity in zip(X, y):
            self.vocabulary[word][entity] += 1

        self.memory = {}
        for key, dictionary in self.vocabulary.items():
            self.memory[key] = max(dictionary, key=dictionary.get)
    
    def predict(self, X, y=None):

        return [self.memory.get(x, 'О') for x in X]

Теперь обучим реализованную модель и посмотрим на ее точность на отложенной выборке для каждой именованной сущности:

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score

X_train = df_train["words"].values.tolist()
X_dev = df_dev["words"].values.tolist()
for name in ["Контрагент", "Город", "ФИО"]:
    y_train = df_train[name].values.tolist()
    y_dev = df_dev[name].values.tolist()
    basic_model = BasicModel()
    basic_model.fit(X_train, y_train)
    print("=====", name, "=====")
    print("precision_score (начало сущности):", precision_score(basic_model.predict(X_dev), y_dev, average=None)[1])
    print("precision_score (продолжение сущности):", precision_score(basic_model.predict(X_dev), y_dev, average=None)[2])
    print("recall_score (начало сущности):", recall_score(basic_model.predict(X_dev), y_dev, average=None)[1])
    print("recall_score (продолжение сущности):", recall_score(basic_model.predict(X_dev), y_dev, average=None)[2])

Видно, что мы получили достаточнос хорошие результаты несмотря на несбалансированность выборки. Однако, такой подход может не сработать в других задачах, когда именованные сущности имеют более сложную структуру.

## Стандартные модели

Прежде чем использовать стандартные методы машинного обученя, нам необходимо каждому объекту из выборки (в нашем случае объекты - это слова) сопоставить признаковое описание. Каждому слову будем сопоставлять следующую информацию:
- длину слова
- является ли слово числом
- содержит ли слово символы, отличные от букв

In [ ]:
def feature_map(word):
    
    return np.array([len(word), word.isdigit(), word.isalpha()])

Преобразуем обучающую и отложенную выборки:

In [ ]:
X_train = [feature_map(w) for w in df_train["words"].values.tolist()]
X_dev = [feature_map(w) for w in df_dev["words"].values.tolist()]

Теперь мы можем применить стандартные модели. Начем со случайного леса и посмотрим на результат его работы для каждой именованной сущности:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

for name in ["Контрагент", "Город", "ФИО"]:
    y_train = df_train[name].values.tolist()
    y_dev = df_dev[name].values.tolist()
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    prediction = model.predict(X_dev)
    print("=====", name, "=====")
    print("precision_score (начало сущности):", precision_score(prediction, y_dev, average=None)[1])
    print("precision_score (продолжение сущности):", precision_score(prediction, y_dev, average=None)[2])
    print("recall_score (начало сущности):", recall_score(prediction, y_dev, average=None)[1])
    print("recall_score (продолжение сущности):", recall_score(prediction, y_dev, average=None)[2])

Видно, что за счет несбалансированности случайный лес обучился выдавать метку $O$ на каждом обращении для каждой сущности.

### Упражнение 1
**Задание:** 
- Обучите случайный лес так, чтобы модель перестала прогнозировать константную метку $O$
- Используйте логистическую регрессию для увеличения значений метрик

Можете использовать следующие способы улучшения результата
- Поскольку выборка является несбалансированной, модель склоняется к предсказанию метки $О$. Чтобы усилить влияние остальных меток при обучении, можно использовать параметр $class\_weight$ при обучении. Например, можно использовать $class\_weight$={$О$ : 1, $н$ : 10, $п$ : 10}, что будет означать, что мы хотим по 10 раз дублировать объекты, метки которых равны $н$ или $п$.
- Попробуйте улучшить признаковое описание слов (функция $feature\_map$). Например, задать условия на окончания слов или на количество гласных и согласных букв в слове.

Переобучите случайный лес с измененными параметрами:

In [ ]:
#Начало кода

#Конец кода

Используйте логистическую регрессию:

In [ ]:
#Начало кода

#Конец кода

**Проверка:**

In [ ]:
assert len(set(lr_prediction)) > 1 and len(set(rf_prediction)) > 1
print("Проверка пройдена!")

# Структурированные методы

Мы начинаем использовать модель, учитывающую порядок слов в предложении, так что старый подход, когда мы анализировали данные по словам и разбивали выборку на обучающую и отложенную не зависимо от их расположения в тексте, уже не подходит. Для каждого типа сущности будем использовать следующее представление данных:
- Разобьем все слова на предложения (учитывая поле $sent\_number$)
- Каждое предложение представим списком пар $(word, entity)$

In [ ]:
sentences = {}

In [ ]:
for name in ["ФИО", "Город", "Контрагент"]:
    get_pair_func = lambda s: [(word, entity) for word, entity in zip(s["words"].values.tolist(),
                                                                      s[name].values.tolist())]
    grouped_words = df.groupby("sent_number").apply(get_pair_func)
    sentences[name] = [sentence for sentence in grouped_words]

In [ ]:
sentences["ФИО"][:1]

Для каждого типа сущности разобьем полученные представления на тренировочную и отложенную выборки:

In [ ]:
from sklearn.model_selection import train_test_split

train_sentences = {}
dev_sentences = {}
for name in ["ФИО", "Город", "Контрагент"]:
    train_sentences[name], dev_sentences[name] = train_test_split(sentences[name], test_size=0.2)
len(train_sentences[name]), len(dev_sentences[name])

Теперь мы можм приступить к применению модели.

## Reccurent Neural Network

Опишем применение рекурентной нейронной сети применительн ок задаче выделеения сущностей. В прошлом практическом задании мы классифицировали документы и использовали сеть типа $Many-to-one$, когда мы проходили по всем словам в обращении и использовали только последюю активацию:

![title](img/2_2.png)

Сейчас же мы должны сопоставить метку каждому слову в обращении, так что актуальна схема $Many-to-many$, когда количество входов нейронной сети совпадает с количеством выходов:

![title](img/2_3.png)

Применим сеть к распознаванию имени, фамилии и отчества:

In [ ]:
name = "ФИО"

Как и раньше, нам необходимо токенизировать обращения и зафиксировать длину обращений. Теперь метки сопоставляются каждом услову, так что их тоже необходимо сделать одинаковой длины. Напишем функции для преобразования обращений и меток:

In [ ]:
MAX_LEN = 50
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

words = list(set(df["words"].values))
tags = list(set(df[name].values))
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {"О" : 0, "н" : 1, "п" : 2}

def sentences2X(sentences):
    X = [[word2idx[w[0]] for w in s] for s in sentences]
    X = pad_sequences(maxlen=MAX_LEN, sequences=X, padding="post",value=len(words) - 1)
    
    return X

def sentences2y(sentences):
    y = [[tag2idx[w[1]] for w in s] for s in sentences]
    y = pad_sequences(maxlen=MAX_LEN, sequences=y, padding="post", value=tag2idx['О'])
    y = [to_categorical(i, num_classes=len(tags)) for i in y]
    
    return np.array(y)

И преобразуем обращения и метки тренировочной и проверочной выборок:

In [ ]:
from keras.preprocessing.sequence import pad_sequences

X_train = sentences2X(train_sentences[name])
X_dev = sentences2X(dev_sentences[name])
y_train = sentences2y(train_sentences[name])
y_dev = sentences2y(dev_sentences[name])

Построим архитектуру нейронной сети:

In [ ]:
import keras.layers as L
from keras.models import Model

layer_input = L.Input(shape=(MAX_LEN,))
layer_emb = L.Embedding(input_dim=len(words), output_dim=MAX_LEN, input_length=MAX_LEN)(layer_input)
layer_drop = L.Dropout(0.1)(layer_emb)
layer_lstm = L.RNN(L.SimpleRNNCell(units=100), return_sequences=True)(layer_drop)
layer_output = L.TimeDistributed(L.Dense(len(tags), activation="softmax"))(layer_lstm)
model = Model(layer_input, layer_output)

In [ ]:
model.summary()

Напишем метрики $Precision$ и $Recall$, которые будем использовать во время обучения:

- $begin\_recall$ - $Recall$ для метки $н$
- $continuous\_recall$ - $Recall$ для метки $п$
- $begin\_precision$ - $Precision$ для метки $н$
- $continuous\_precision$ - $Precision$ для метки $п$

In [ ]:
import tensorflow as tf

EPS=1e-10
def begin_recall(y_true, y_pred):
    
    return tf.reduce_sum(y_true[::, ::, 1] * y_pred[::, ::, 1]) / (tf.reduce_sum(y_true[::, ::, 1]) + EPS)
def continuous_recall(y_true, y_pred):
    
    return tf.reduce_sum(y_true[::, ::, 2] * y_pred[::, ::, 2]) / (tf.reduce_sum(y_true[::, ::, 2]) + EPS)
def begin_precision(y_true, y_pred):
    
    return tf.reduce_sum(y_true[::, ::, 1] * y_pred[::, ::, 1]) / (tf.reduce_sum(y_pred[::, ::, 1]) + EPS)
def continuous_precision(y_true, y_pred):
    
    return tf.reduce_sum(y_true[::, ::, 2] * y_pred[::, ::, 2]) / (tf.reduce_sum(y_pred[::, ::, 2]) + EPS)

Скомпилируем модель:

In [ ]:
model.compile(optimizer="adam", 
              loss="categorical_crossentropy", 
              metrics=[begin_recall, continuous_recall, begin_precision, continuous_precision])

И обучим:

In [ ]:
model.fit(X_train, np.array(y_train), 
          batch_size=16, 
          epochs=5,
          validation_data = (X_dev, np.array(y_dev)),
          verbose=1)

### Упражнение 2

**Задание:** Реализуйте собственную архитектуру рекуррентной нейронной сети и попытайтесь превзойти качество простой модели. Рекомендации к реализации:
- Попробуйте варьировать размерность вектора активации сети
- Вместо $RNN$ ячейки используйте ячейки $LSTM$ или $BiLSTM$ ($L.LSTM$, $L.Bidirectional(L.LSTM)$)
- Попробуйте добавить несколько полносвязных слоев к активации с каждого слова

Реализуйте вашу модель:

In [ ]:
#Начало кода


#Конец кода

Скомпилируйте:

In [ ]:
model.compile(optimizer="adam", 
              loss="categorical_crossentropy", 
              metrics=[begin_recall, continuous_recall, begin_precision, continuous_precision])

И обучите:

In [ ]:
history = model.fit(X_train, np.array(y_train), 
          batch_size=16, 
          epochs=5,
          validation_data = (X_dev, np.array(y_dev)),
          verbose=1)

**Проверка:**

In [ ]:
assert history.history['val_begin_recall'][-1] > 0.62 and history.history['val_continuous_recall'][-1] > 0.75
print("Результат простой модели превзойден!")